In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')

from DSProject import DSProject
from preprocess import *

dsp = DSProject()

In [ ]:
# load txn_cc
j=0
for chunk in pd.read_csv(os.path.join(dsp.data_directory, 'EDW_SOR_TXN_CC.txt'), delimiter='|', header=None, usecols=[0,1,9,13,18], chunksize=1000000):
    j = j+1
    chunk.columns = ['HASH_IP_ID','HASH_AR_ID','TXN_CODE','TXN_VAL_DT','NET_CASH_FLOW_AMT']
    txn_cc = chunk.loc[chunk['TXN_CODE'].isin([19,20])]
    dsp.write_table(txn_cc, 'SOR_TXN_CC_filtered_' + str(j), 'preprocess', gen_schema=False)

In [2]:
chunks = []
for i in range(0, 55):
    chunk = dsp.read_table('SOR_TXN_CC_filtered_' + str(i), 'preprocess', index_col=0)
    chunks.append(chunk)
df = pd.concat(chunks)

In [3]:
df['TXN_VAL_DT'] = parse_dates(df['TXN_VAL_DT'])
unique_filedate = dsp.read_table('unique_filedate', 'feature', index_col=0, parse_dates=['FileDate', 'Next_FileDate', 'Previous_FileDate'])
df = df.merge(unique_filedate, on=['HASH_AR_ID'])

In [4]:
df = df.loc[(df['TXN_VAL_DT'] >= df['FileDate']) & (df['TXN_VAL_DT'] < df['Next_FileDate'])]

In [5]:
payment = df.groupby(['HASH_AR_ID', 'FileDate'])['NET_CASH_FLOW_AMT'].sum().reset_index()
payment.columns = ['HASH_AR_ID', 'FileDate', 'Payment']

In [6]:
payment['HASH_AR_ID'] = payment['HASH_AR_ID'].astype(str)

In [7]:
dsp.write_table(payment, 'payment', 'feature')

In [8]:
# generate payment percentage
# load group_target and merge

group_target = dsp.read_table('group_target', 'feature', index_col=0, use_schema=True)
df = pd.merge(group_target, payment, on=['HASH_AR_ID', 'FileDate'], how ='left')

In [9]:
print("Merge coverage")
print(len(df.index)/len(group_target.index))

Merge coverage
1.0


In [10]:
df['Payment'] = df['Payment'].fillna(0)

In [11]:
df['Percentage'] = df['Payment']/df['OSAMT']

In [12]:
df['Percentage'].loc[df['Percentage']>1.0] = 1.0

In [13]:
df = df[['HASH_AR_ID', 'HASH_IP_ID', 'HASH_LPM_CST_ID', 
         'FileDate', 'Previous_FileDate', 'Next_FileDate', 
         'Group', 'OSAMT', 'Payment', 'Percentage']]

In [14]:
dsp.write_table(df, 'group_payment_target', 'feature', 
                ds_type=['id', 'id', 'id', 'date', 'date', 'date', 
                         'category', 'numeric', 'numeric', 'numeric'])

In [15]:
print(len(df.index))

1078399
